In [ ]:
import json
import time
import random
TIME_SCALE = 50 # 300x faster than real time
BASE_TICK_SECONDS = 2

# ---------------- DATA LOADER ----------------
class DataLoader:
    def __init__(self, path):
        with open(path, "r") as f:
            self.data = json.load(f)

# ---------------- PLATFORM ----------------
class Platform:
    def __init__(self, pid, door_side):
        self.platform_id = pid
        self.door_side = door_side
        self.is_occupied = False
        self.occupied_by = None

# ---------------- STATION ----------------
class Station:
    def __init__(self, name, platforms):
        self.name = name
        self.platforms = {
            pid: Platform(pid, info["door_side"])
            for pid, info in platforms.items()
        }

    def get_random_free_platform(self):
        free = [p for p in self.platforms.values() if not p.is_occupied]
        return random.choice(free) if free else None

# ---------------- TRAIN ----------------
class Train:
    def __init__(self, tid, capacity, speed):
        self.train_id = tid
        self.capacity = capacity
        self.speed = speed
        self.status = "STOPPED"

# ---------------- SIGNAL SYSTEM ----------------
class SignalSystem:
    @staticmethod
    def request_platform(platform, train_id):
        if platform.is_occupied:
            return False
        platform.is_occupied = True
        platform.occupied_by = train_id
        return True

    @staticmethod
    def release_platform(platform):
        platform.is_occupied = False
        platform.occupied_by = None

# ---------------- SIMULATION ENGINE ----------------
class SimulationEngine:

    def move_segment(self, train, distance):
        remaining = distance
        next_print_mark = distance - 2  # print every 2 km

        while remaining > 0:
            # move train every 2 seconds
            covered = train.speed * (2 / 3600)
            remaining -= covered
            remaining = max(0, remaining)

            # print only at 2 km milestones
            if remaining <= next_print_mark or remaining == 0:
                completed = distance - remaining
                print(
                    f"Train {train.train_id} | "
                    f"Completed: {completed:.0f} km | "
                    f"Remaining: {remaining:.2f} km"
                )
                next_print_mark -= 2

            time.sleep(BASE_TICK_SECONDS / TIME_SCALE)


    def run_route(self, train, route, stations, tracks):
        for i in range(len(route) - 1):
            curr = route[i]
            nxt = route[i + 1]
            track_key = f"{curr}-{nxt}"
            distance = tracks[track_key]["distance_km"]

            print(f"\n[DEPARTURE] Leaving {curr}")
            self.move_segment(train, distance)

            station = stations[nxt]
            platform = station.get_random_free_platform()

            while platform is None:
                print(f"[WAIT] All platforms occupied at {nxt}")
                time.sleep(3)
                platform = station.get_random_free_platform()

            SignalSystem.request_platform(platform, train.train_id)

            print(f"""
[ARRIVAL]
Station: {nxt}
Platform: {platform.platform_id}
Doors will open on: {platform.door_side} side
Train Capacity: {train.capacity}
Next Station: {route[i+2] if i+2 < len(route) else "END"}
""")

            time.sleep(3)
            SignalSystem.release_platform(platform)

# ---------------- USER INPUT ----------------
def choose_station(stations, prompt):
    print(f"\n{prompt}")
    for i, s in enumerate(stations, 1):
        print(f"{i}. {s}")
    return stations[int(input("Enter choice number: ")) - 1]

# ---------------- MAIN ----------------
if __name__ == "__main__":
    loader = DataLoader("data.json")

    stations = {
        name: Station(name, info["platforms"])
        for name, info in loader.data["stations"].items()
    }

    route = loader.data["routes"]["WesternLine"]

    # User selects source & destination
    src = choose_station(route, "Select SOURCE station")
    dst = choose_station(route, "Select DESTINATION station")

    if src == dst:
        print("[ERROR] Source and destination cannot be the same")
        exit()

    src_idx = route.index(src)
    dst_idx = route.index(dst)

    # Determine direction
    if src_idx < dst_idx:
        active_route = route[src_idx:dst_idx + 1]
    else:
        active_route = list(reversed(route[dst_idx:src_idx + 1]))

    print(f"\n[ROUTE] {' → '.join(active_route)}")

    # Train creation
    t = loader.data["trains"]["T2"]
    train = Train("T2", t["capacity"], t["speed_kmph"])

    engine = SimulationEngine()
    engine.run_route(train, active_route, stations, loader.data["tracks"])


In [ ]:
{
  "stations": {
    "Virar": {
      "platforms": {
        "P1": { "door_side": "LEFT", "capacity": 1400 },
        "P2": { "door_side": "RIGHT", "capacity": 1200 }
      }
    },
    "Vasai": {
      "platforms": {
        "P1": { "door_side": "RIGHT", "capacity": 1300 },
        "P2": { "door_side": "LEFT", "capacity": 1500 }
      }
    },
    "Dadar": {
      "platforms": {
        "P3": { "door_side": "LEFT", "capacity": 2000 },
        "P4": { "door_side": "RIGHT", "capacity": 1800 }
      }
    },
    "Churchgate": {
      "platforms": {
        "P1": { "door_side": "LEFT", "capacity": 2500 },
        "P2": { "door_side": "RIGHT", "capacity": 2200 }
      }
    }
  },

  "routes": {
    "WesternLine": ["Virar", "Vasai", "Dadar", "Churchgate"]
  },

  "tracks": {
    "Virar-Vasai": { "distance_km": 12 },
    "Vasai-Dadar": { "distance_km": 35 },
    "Dadar-Churchgate": { "distance_km": 9 }
  },

  "trains": {
    "T1": { "capacity": 1600, "speed_kmph": 60 },
    "T2": { "capacity": 1400, "speed_kmph": 70 }
  }
}
